![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# An Introduction to Simple SAR Change Detection Methods
<img src="NotebookAddons/UAFLogo_A_647.png" style="padding:7px;" width="170" align="right" />

### Franz J Meyer; University of Alaska Fairbanks

This notebook introduces you to a some popular change detection methods that can be applied to SAR time series data. SAR is an excellent tool for change detection. As SAR sensors are weather and illumination independent and carry their own illumination source (active sensor). Differences between repeated images are a direct indication of changes on the surface. This fact is exploited by the change detection methods introduced below. 
    
The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis.

<img style="padding:7px;" src="NotebookAddons/OpenSARlab_logo.svg" width="170" align="right" />

**This notebook covers the following data analysis concepts:**

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference thresholding
- Time series coefficient of variation thresholding
- Log Ratio-based change detection from image pairs

---
**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

---
## 0. Importing Relevant Python Packages

In this notebook we will use the following scientific libraries:

- [Pandas](https://pandas.pydata.org/) is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality.
- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
- [NumPy](http://www.numpy.org/) is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.
- [Matplotlib](https://matplotlib.org/index.html) is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib.
- [SciPy](https://www.scipy.org/about.html) is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics.

**Our first step is to import them:**

In [ ]:
%%capture
import datetime # for date
from pathlib import Path
from os import system

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc


from IPython.display import HTML

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

---
## 1. Load Data Stack

<img src="NotebookAddons/mapfuego.png" width="250" style="padding:5px;" align="right" /> 

This notebook will use a dense 12-day repeat Sentinel-1 C-band SAR data stack. It contains imagery acquired during the year 2018 in Guatemala. The data are already prepared for you. In a later lecture, you will learn how to download and pre-process Sentinel-1 images using the services of the [Alaska Satellite Facility](https://www.asf.alaska.edu/). 

**2018 Volcán de Fuego eruption**

On June 03, 2018, Volcán de Fuego in Guatemala erupted suddenly. 190 people lost their lives. Eruptions continued in June, and November saw a phase of renewed activity. The **pyroclastic flows and lahars** (mudflow) associated with the eruptive episodes also caused great damage to property and crops. 

 <table><tr>
<td> <img src="NotebookAddons/fuegoash.jpg" style="width: 300px;"/> </td>
<td> <img src="NotebookAddons/fuegoash2.jpg" style="width: 300px;"/> </td>
</tr></table>

*Getty Images*

We will use the Sentinel-1 data stack to map land surface changes throughout this period. 

Before we get started, let's first **create a working directory for this analysis and change into it:**

In [ ]:
name = 'fuego'
path = Path(f'/home/jovyan/notebooks/SAR_Training/English/Hazards/{name}')

if not path.exists():
    path.mkdir()

We will **retrieve the relevant data** [Amazon Web Service (AWS)](https://aws.amazon.com/) cloud storage bucket **using the following command:**

In [ ]:
time_series_path = 's3://asf-jupyter-data/fuego.tar.gz'
time_series = path/Path(time_series_path).name
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

Now, let's **unzip the file (overwriting previous extractions) and clean up after ourselves:**

In [ ]:
!tar -xvzf {time_series} -C {path}

## 2. Define Some Python Helper Functions for this Notebook

We are defining two helper functions for this notebook:

- **CreateGeoTiff()** to write out images
- **timeseries_metrics()** to compute various metrics from a time series data stack

In [ ]:
def create_geotiff(name, array, data_type, ndv, bandnames=None, 
                   ref_image=None, geo_t=None, projection=None):
    # If it's a 2D image we fake a third dimension:
    if len(array.shape) == 2:
        array = np.array([array])
    if ref_image == None and (geo_t == None or projection == None):
        raise RuntimeWarning('ref_image or settings required.')
    if bandnames != None:
        if len(bandnames) != array.shape[0]:
            raise RuntimeError(f'Need {Array.shape[0]} bandnames. {len(bandnames)} given')
    else:
        bandnames = [f'Band {i+1}' for i in range(array.shape[0])]
    if ref_image != None:
        if not isinstance(ref_image, str):
            ref_image = str(ref_image)
            
        refimg = gdal.Open(ref_image)
        geo_t = refimg.GetGeoTransform()
        Projection = refimg.GetProjection()
    driver = gdal.GetDriverByName('GTIFF')
    array[np.isnan(array)] = ndv
    dataset = driver.Create(name, array.shape[2], array.shape[1], 
                            array.shape[0], data_type)
    dataset.SetGeoTransform(geo_t)
    dataset.SetProjection(projection)
    for i, image in enumerate(array, 1):
        dataset.GetRasterBand(i).WriteArray(image)
        dataset.GetRasterBand(i).SetNoDataValue(ndv)
        dataset.SetDescription(bandnames[i-1])
    dataset.FlushCache()
    return name

In [ ]:
def timeseries_metrics(raster, ndv=np.nan): 
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float')>-1:
        raster = raster.astype(np.float64)
    # Set ndv to nan
    if not np.isnan(ndv):
        raster[np.equal(raster,ndv)] = np.nan
    # Build dictionary of the metrics
    tsmetrics={}
    rperc = np.nanpercentile(raster,[5,50,95], axis=0)
    tsmetrics['mean'] = np.nanmean(raster, axis=0)
    tsmetrics['max'] = np.nanmax(raster, axis=0)
    tsmetrics['min'] = np.nanmin(raster, axis=0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2]-rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis=0)
    tsmetrics['std'] = np.sqrt(tsmetrics['var'])
    tsmetrics['CV'] = np.abs(tsmetrics['var'] / tsmetrics['mean'])
    return tsmetrics

## 3. Define Data Directory and Path to VRT

**Create a variable containing the VRT filename and the image acquisition dates:**

In [ ]:
pol = 'VH' # you can also try 'VV' later
imagefile = path/f'stack{name}_{pol}.vrt'

**Create an index of timedelta64 data with Pandas:**

In [ ]:
datefile = path/f'dates{name}_{pol}.csv'
tindex = pd.DatetimeIndex(open(datefile).read().split(','))

**Print the bands and dates for all images in the virtual raster table (VRT):**

In [ ]:
j = 1
print(f"Bands and dates for {imagefile}")
for i in tindex:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

---
## 4. Create a Time Series Animation to get an Idea of the Dynamics at the Site

### 4.1 Load Time Series Stack

Now we are ready to create a time series animation from the calibrated SAR data.

**First, create a raster from band 0 and a raster stack from all the images:**

In [ ]:
img = gdal.Open(str(imagefile))
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack = img.ReadAsArray()

In [ ]:
print(rasterstack.shape)

**Print the bands, pixels, and lines:**

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

## 4.2 Data Conversion between dB and Power Scales

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^0$ is the preferred metric.

To use a logarithmic scale instead of the natural power scale, **you can set the following variable to True:**

In [ ]:
todB = True

labeldB = 'dB' if todB else 'linear'

def convert(raster, todB=todB):
    if todB:
        return 10 * np.ma.log10(raster)
    else:
        return raster

## 4.3 Create Time Series Animation

**Create and move into a directory in which to store our plots and animations:**

In [ ]:
product_path = path/'plots_and_animations'

if not product_path.exists():
    product_path.mkdir()

Now we can **create the information needed to animate our data:**

In [ ]:
%%capture 
fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()
ax.axis('off')
vmin = np.nanpercentile(convert(rasterstack), 1)
vmax = np.nanpercentile(convert(rasterstack), 99)

im = ax.imshow(convert(raster0), cmap='inferno', vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im)
cbar.set_label(labeldB)
ax.set_title("{}".format(tindex[0].date()))
plt.rcParams.update({'font.size': 14})

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(convert(rasterstack[i]))

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack.shape[0], interval=300)

**Configure matplotlib's RC settings for the animation:**

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

**Create a javascript animation of the time-series running inline in the notebook:**

In [ ]:
HTML(ani.to_jshtml())

**Delete the dummy png** that was saved to the current working directory while generating the javascript animation in the last code cell.

In [ ]:
try:
    Path('None0000000.png').unlink()
except FileNotFoundError:
    pass

**Save the animation (animation.gif):**

In [ ]:
ani.save(product_path/f'animation_{labeldB}.gif', writer='pillow', fps=2)

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #1</u>:  </font> Can You See the Impact of the Eruption?</b> </font>

<font face="Calibri" size="3"> Can you see the impact of the ongoing eruption in the SAR images? Can you identify the lahar flows? When did they occur and how far did they reach?
</font>
</div>

---
## 5. Computation and Visualization of Time Series Metrics

Once a time-series was constructed, we can compute **a set of metrics** for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

---
First, we **mask out pixels** that are zero (e.g. beyond the edge of the swath). Then we **calculate the time series metrics:**

In [ ]:
mask = (rasterstack == 0)
raster = np.ma.array(convert(rasterstack), mask=mask, dtype=np.float64)

In [ ]:
%%capture
metrics = timeseries_metrics(raster.filled(np.nan), ndv=np.nan)

In [ ]:
set(metrics.keys())

Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=np.nanpercentile(metrics['var'], [1,99]))
ax[1].hist(metrics['CV'].flatten(), bins=100, range=np.nanpercentile(metrics['CV'], [1,99]))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

In [ ]:
# List the metrics keys you want to plot
metric_keys=['mean', 'median', 'max', 'min', 
             'p95', 'p5', 'prange', 'var', 'std', 'CV']
fig= plt.figure(figsize=(16,40))
idx=1
for i in metric_keys:
    ax = fig.add_subplot(5,2,idx)
    vmin, vmax = np.nanpercentile(metrics[i], [1, 99])
    ax.imshow(metrics[i],vmin=vmin,vmax=vmax,cmap='inferno')
    ax.set_title(i.upper())
    ax.axis('off')
    idx+=1

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #2:</u> </font></b> 

<font face="Calibri" size="3"> Can you group the metrics in terms of what they are most sensitive to? How would you expect the sensitivity of each metric to change if you switched from dB to linearly scaled images?
</font></font>
</div>

You might have noticed white patches in the images above. These do not contain any data. The reason is that they are in the radar shadow of terrain that is closer to the satellite.

You might have noticed white patches in the images above. These do not contain any data. The reason is that they are in the radar shadow of terrain that is closer to the satellite.

---

## 6. Some Popular SAR Change Detection Methods

This section will introduce you to the following popular and simple change detection methods:

- Time series metrics  95$^{th}$ and 5$^{th}$ percentile difference and standard deviation thresholding
- Time series coefficient of variation thresholding

---
### 6.1 Change Detection with the Percentile Difference and the Variance Threshold Method

In this method we find thresholds on the **95$^{th}$ and 5$^{th}$ percentile difference** or the **temporal pixel-by-pixel gray value cariance**. Let's start with the 95$^{th}$ and 5$^{th}$ percentile difference. The advantage to look at percentiles verus maximum minus minimum is that it is more robust to outliers.

First, let us define a **function for plotting histograms**

In [ ]:
def plot_histogram_cdf(metric):
    plt.rcParams.update({'font.size': 12})
    fig = plt.figure(figsize=(14, 4)) # Initialize figure with a size
    ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
    ax2 = fig.add_subplot(122)

    h = ax1.hist(
        metrics[metric].flatten(), bins=200, range=np.nanpercentile(metrics[metric], [1, 99]))
    ax1.xaxis.set_label_text(f'{metric} {labeldB}')
    ax1.set_title('Histogram')

    n, bins, patches = ax2.hist(
        metrics[metric].flatten(), bins=200, range=np.nanpercentile(metrics[metric], [1, 99]),
        cumulative='True', density='True', histtype='step', label='Empirical')
    ax2.xaxis.set_label_text(f'{metric} {labeldB}')
    ax2.set_title('CDF')

    outind = np.where(n > 0.95)
    threshind = np.min(outind)
    thresh = bins[threshind]
    ax1.axvline(thresh,color='red')
    _ = ax2.axvline(thresh,color='red')
    plt.savefig(product_path/f'{metric}_{labeldB}_histogram.png',
            dpi=200, transparent='true')

**Now let's look at the 95th - 5th percentile range**

In [ ]:
plot_histogram_cdf(metric='prange')

Let's visualize the 5% of all pixels with the largest (95th - 5th percentile) difference in the time series. We will refer to the pixels (x,y) that exceed this threshold $t$ as likely **change pixels (cp):**

${cp}_{x,y} = P_{x,y}^{95th} - P_{x,y}^{5th} > t$ 

If we define $t$ to correspond to the 5% of pixels with highest (95th - 5th percentile) difference, the image looks like:

In [ ]:
def plot_threshold_classifier(metric='prange', percentage_cutoff=5):
    plt.figure(figsize=(8,8))
    thresh = np.nanpercentile(metrics[metric], 100 - percentage_cutoff)
    mask = metrics[metric] < thresh # For display we prepare the inverse mask
    plt.imshow(mask, cmap='gray')
    _=plt.title(f'Threshold Classifier on {metric} > %1.3f' % thresh)
    plt.savefig(product_path/f'changes_{metric}_{labeldB}.png',
                dpi=200, transparent='true')
    return np.logical_not(mask)

metric = 'prange'
masks = {metric: plot_threshold_classifier(metric=metric)}

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #3:</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure. What are the black areas? What are the white areas? What kind of changes may be included in this map?
</font></font>
</div>

Instead of applying a threshold on the 95th - 5th percentile difference data, we can also attempt to threshold other metrics. The **standard deviation** (or variance) variable seems a useful indicator for change as it identifies pixels for which radar brightness has changed strongly within the time series. Hence, in the following we use this metric for change identification according to:

${cp}_{x,y} = \sigma > t$ 

with $t=CDF_{\sigma} > 0.95$ (5% pixels with highest standard deviation):

In [ ]:
plot_histogram_cdf(metric='std')

In [ ]:
metric = 'std'
masks[metric] = plot_threshold_classifier(metric=metric)

## 6.2 Change Detection with the Coefficient of Variation Method

We can also set a threshold $t$ for the **coefficient of variation image**
to classify change in the time series:
    
${CV}_{x,y} = \frac{\sigma_{x,y}}{\overline{X}_{x,y}} > t$ 

Let's look at the histogram and the Cumulative Distribution Function (CDF) of the coefficient of variation:

In [ ]:
plot_histogram_cdf(metric='CV')

With a threshold of $t=CDF_{CV} > 0.95$ (5% pixels with highest variance) the change pixels would look like the following image:

In [ ]:
metric = 'CV'
masks[metric] = plot_threshold_classifier(metric=metric)

## 6.3 Pair-wise change detection

To analyze temporal changes between two images, it is useful to compute metrics that are sensitive to discrepancies between the two images. In radar remote sensing, the standard way is to look at ratios (in the linearly scaled power domain) or, equivalently, at differences in the logarithmic dB domain. 

In [ ]:
dates = ('2018-05-27', '2018-06-08') # around first eruption

# convert to datetime objects
dates_ = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# get the indices in one line
dates_ind = [np.argmin(np.abs(date - tindex)) for date in dates_]
print(f'Comparing image {dates_ind[0]} from {tindex[dates_ind[0]].date()} with {dates_ind[1]} from {tindex[dates_ind[1]].date()}')

**Compute the log ratio in dB**, corresponding to the difference in dB

In [ ]:
ratiodB = 10 * np.ma.log10(np.ma.divide(rasterstack[dates_ind[1], ...], rasterstack[dates_ind[0], ...]))

**Let us manually choose a threshold this time**

thresh is the threshold, e.g. -2 dB
thresh_type determines whether we mask everything below that (lower) or above (upper)

In [ ]:
thresh = -2
thresh_type = 'lower' #'lower': mask everything below thresh, 'upper': mask everything above

dynamic_range = np.nanpercentile(np.abs(ratiodB), 99)
fig, axs = plt.subplots(ncols=3, nrows=1)
fig.suptitle(f'{tindex[dates_ind[0]].date()} to {tindex[dates_ind[-1]].date()}     ', ha='right', fontsize='15')
fig.set_size_inches(20, 4)
plt.subplots_adjust(hspace=0.4, right=0.85)
h = axs[0].hist(
        ratiodB.flatten(), bins=200, range=np.nanpercentile(ratiodB, [0.1, 99.9]))
axs[0].xaxis.set_label_text(f'difference [dB]')
axs[0].set_title('Histogram')
mask = (ratiodB > thresh if thresh_type == 'lower' else ratiodB < thresh).astype(np.int8)
axs[1].imshow(mask, cmap='gray')
axs[1].set_title('Mask')
im0 = axs[2].imshow(ratiodB, cmap='RdBu', vmin=-dynamic_range, vmax=dynamic_range)
cbar = fig.colorbar(im0, orientation='vertical', ax=axs.ravel().tolist())
cbar.set_label('[dB]')
axs[2].set_title('Image')
logratiolabel = f'logratio_{tindex[dates_ind[0]].date()}_{tindex[dates_ind[0]].date()}'
plt.savefig(product_path/f'{logratiolabel}.png',
            dpi=200, transparent='true')
masks[logratiolabel] = np.logical_not(mask)

**Do you think the threshold is appropriate?**

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #4:</u> </font></b> 

<font face="Calibri" size="3"> Discuss what you see in this figure: 
    <ul>
        <li>What kind of changes are detected?</li>
        <li>How does this change map compare to the previous one? Note that you are only looking at one specific pair of images here.</li>
        <li>Feel free to look at other image pairs and analyze identified changes!</li>
    </ul>
    </font></font>
</div>

---
## 7. Write Our Change Detection Results and Metrics Images to GeoTIFF files

### 7.1 Determine Output Geometry

First, we need to **set the correct geotransformation and projection information**. We retrieve the values from the input images:

In [ ]:
proj = img.GetProjection()
geotrans = list(img.GetGeoTransform())
geotrans

### 7.2 Output Time Series Metrics Images

We use the root of the time series data stack name and append a _ts_metrics_&lt;metric&gt;.tif ending as filenames:

In [ ]:
# Time Series Metrics as image image:
# We make a new subdirectory where we will store the images
dirname = path/f'{name}_tsmetrics'

if not dirname.exists():
    dirname.mkdir()
    
print(dirname)

Now we can **output the individual metrics as GeoTIFF images:**

In [ ]:
names=[] # List to keep track of all the names
for metric in metrics:
    name_ = str(dirname/f'{metric}_{labeldB}.tif')
    create_geotiff(name_,metrics[metric], gdal.GDT_Float64, np.nan,[metric],
                  geo_t=geotrans, projection=proj)
    names.append(name_)

### 7.3 Build a Virtual Raster Table on the Metrics GeoTIFF images

To tie the images into one new raster stack of time series metrics we build a virtual raster table with all the metrics. 

Trick: Use ' '.join(names) to build one long string of names separated by a space as input to *gdalbuildvrt*:

In [ ]:
cmd='gdalbuildvrt -separate -overwrite -vrtnodata nan '+\
   f'{dirname}_{labeldB}.vrt '+\
    ' '.join(names)
# print(cmd)
_ = system(cmd)

### 7.4 Create GeoTIFFs for the Change Images from our Four Change Detection Attempts

We are going to write GeoTIFF output files that store the results from the classifiers:

In [ ]:
for metric in masks:
    fnmetric = str(dirname/f'{name}_{labeldB}_{metric}_thresholds.tif')
    create_geotiff(fnmetric, masks[metric], gdal.GDT_Byte, np.nan, 
                  geo_t=geotrans, projection=proj)


<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #5:</u> </font></b> </font>

<font face="Calibri" size="3"> Explore the data stack a bit more. Answer the following questions for yourself:

<ul>
  <li> Which metrics are more appropriate for logarithic (dB) or linearly (power) transformed images?</li>
  <li> Change the thresholds and dates in this notebook to see the effects on detected changes.</li>
  <li> Load created change masks into QGIS and compare the detected areas with your time series plots and image data.</li>
</ul>
</font>
</div>

<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE #6:</u> </font></b></font>

<font face="Calibri" size="4"><b>2016-2017 flooding</b></font><br>
<font face="Calibri" size="3"><br>Explore the flooding data stack from Lab 3 at VV and VH polarization. What can you detect? Compare the minimum method for flood detection from Lab 3 with the results obtained with this notebook.
<br><br>
To this end, we advise you to restart the notebook (or duplicate it first) by going to 'Kernel' and 'Restart'
</font>
</div>

*Exercise4A-SARChangeDetectionMethods - Version 1.5.0 - November 2021*

*Version Changes:*

- *asf_notebook -> opensarlab_lib*
- *url_widget*
- *html -> markdown*